In [21]:
import json
import os
import sys

import numpy as np
import pandas as pd
from tqdm import tqdm

sys.path.append(os.path.abspath(".."))

from config import (
    TRAIN_DATA_PATH,
    TEST_DATA_PATH,
    METADATA_PATH,
    EMBEDDINGS_TITLE_PATH,
    EMBEDDINGS_DESCR_PATH,
    EMBEDDINGS_COMBI_PATH,
    N_COMPONENTS,
    RANDOM_STATE,
    N_RECOMMENDATIONS,
    N_EPOCHS,
    EPSILON,
    RECOMMENDATIONS_PATH,
    TRANSFORMED_DATA_PATH,
)
import utils

In [2]:
# importing training data
train_df = pd.read_parquet("../" + TRAIN_DATA_PATH)

# importing test data
test_df = pd.read_parquet("../" + TEST_DATA_PATH)

# importing metadata
meta_df = pd.read_parquet("../" + METADATA_PATH)

# importing the transformed data
transformed_df = pd.read_parquet("../" + TRANSFORMED_DATA_PATH)

In [29]:
# loading embeddings
emb_df = pd.read_parquet("../" + EMBEDDINGS_TITLE_PATH)
emb_df.drop(columns="episode", inplace=True)
min_value = emb_df.min().min()
max_value = emb_df.max().max()

print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")

Minimum value: -1.4321266412734985
Maximum value: 2.9307823181152344


In [30]:
# loading embeddings
emb_df = pd.read_parquet("../" + EMBEDDINGS_DESCR_PATH)
emb_df.drop(columns="episode", inplace=True)
min_value = emb_df.min().min()
max_value = emb_df.max().max()

print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")

Minimum value: -1.4321266412734985
Maximum value: 2.5043904781341553


In [31]:
# loading embeddings
emb_df = pd.read_parquet("../" + EMBEDDINGS_COMBI_PATH)
emb_df.drop(columns="episode", inplace=True)
min_value = emb_df.min().min()
max_value = emb_df.max().max()

print(f"Minimum value: {min_value}")
print(f"Maximum value: {max_value}")

Minimum value: -1.4321266412734985
Maximum value: 2.5043904781341553


In [10]:
train_and_test_df = pd.concat([train_df, test_df])
episode_counts = train_and_test_df.groupby("prd_number").size().reset_index(name="count")
top_10_episodes_tt = episode_counts.nlargest(10, "count")["prd_number"].tolist()
top_10_episodes_tt

['11802450402',
 '16122492630',
 '16122492637',
 '16122493781',
 '16122492636',
 '16122493791',
 '16122492631',
 '16122492639',
 '16122492633',
 '16122492632']

In [20]:
my_set = {}
my_list = ["hund", "høne"]

bool(my_set)

False

In [11]:
episode_counts = transformed_df.groupby("prd_number").size().reset_index(name="count")
top_10_episodes_tr = episode_counts.nlargest(10, "count")["prd_number"].tolist()
top_10_episodes_tr

['11802450402',
 '16122492630',
 '16122493781',
 '16122493791',
 '16122492631',
 '16122492636',
 '16122492637',
 '16122492632',
 '16122492633',
 '16122492639']